In [0]:
%pwd

'/content'

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import math
import statistics as s


### 데이터 적재

In [0]:
# 행정동 추가 파일 불러오기
airdef = pd.read_excel('C:/Python/JupyterNotebook/[공모전]빅콘/DATASET(미세먼지)/04_Innovation 분야_환경기상데이터(케이웨더)_데이터정의서(행정동추가)_190726.xlsx', sheet_name = '경진대회용 후보 지점')
airdef.drop('Unnamed: 5', axis=1, inplace=True)
airdef.head()

FileNotFoundError: ignored

In [0]:
j_hdong = airdef.loc[:,('스테이션','행정동')]
j_hdong.set_index('행정동', inplace=True)
j_hdong

In [0]:
n_hdong = airdef.loc[:,('스테이션.1','행정동.1')].dropna(axis=0, how='all')
#n_hdong.set_index('행정동.1', inplace=True)
n_hdong

In [0]:
print('종로구 내 행정동 갯수 :', len(airdef['행정동'].unique()),'\n노원구 내 행정동 갯수 :',len(airdef['행정동.1'].unique()))

In [0]:
# 종로구 내 행정동
airdef['행정동'].value_counts().sort_index()

In [0]:
# 노원구 내 행정동
airdef['행정동.1'].value_counts().sort_index()

In [0]:
jong_dong = sorted(airdef['행정동'].unique())
jong_dong

In [0]:
no_dong = sorted(n_hdong['행정동.1'].unique())
no_dong

In [0]:
j_hdong.loc['가회동',:].values

In [0]:
n_hdong.set_index('행정동.1', inplace=True)
n_hdong.loc['공릉1동',:].values

In [0]:
n_hdong.loc['하계1동',:].values

In [0]:
tm_all = pd.DataFrame(data = pd.date_range(start='2018-04-01 00:00:00', end='2019-03-31 23:59:00',freq='1min')\
             .strftime('%Y%m%d%H%M').astype(np.int64), columns = ['tm'])

In [0]:
tm_all.head()

In [0]:
len(tm_all)

### [함수] 1개 행정동 내 여러 관측소 매핑 함수

In [0]:
def multi_st(num):
    df_return = tm_all
    for i in n_hdong.loc[no_dong[num],:].values:  # 여기서 no_dong[_] 안의 숫자만 바꿔가면서 돌려주면 됨
        filepath = 'C:\Python\JupyterNotebook\[공모전]빅콘\DATASET(미세먼지)\환경기상데이터\노원구\{}.csv'.format(i[0])
        a = pd.read_csv(filepath)
        b = a['pm10'].replace([-999,-9999], np.NaN)
        b = a['noise'].replace([-999,-9999], np.NaN)
        b = a['temp'].replace([-999,-9999], np.NaN)
        b = a['humi'].replace([-999,-9999], np.NaN)
        b = a['pm25'].replace([-999,-9999], np.NaN)
        b.dropna(how='all',inplace=True)
        if len(b) == 0:
            pass
        else:
            df_return = pd.merge(df_return,a[['tm', 'pm10', 'pm25', 'temp', 'humi']],on = 'tm',how = 'outer')
    df_return['DONG_NM'] = no_dong[num]
    return df_return
    


### [함수] 최빈값(mode) 구하는 함수

In [0]:
# 최빈값 구하는 함수

def mode(gugan_num):
    
    d = {}
    num = list(map(str, gugan_num))
    
    for c in num:
        if c in d:
            d[c] += 1
        else:
            d[c] = 1
    
    return max(d, key=d.get)

## 1.공릉1동(station 2개)

In [0]:
# 행정동 - nowon1(공릉1동) - station 2개
# 행정동별 datetime변경 및 이상치 처리 

노원행정동1 = multi_st(1)

# tm 변수를 시간 변수로 바꾸고 인덱스로 설정해주기
노원행정동1['tm'] = pd.to_datetime(노원행정동1['tm'], format = '%Y%m%d%H%M')


#하고 싶은 것 :   
#동별 시간대별 미세먼지 농도
# -999, -9999 na로 바꾸고 해보자

노원행정동1['pm10_x'].replace([-999,-9999], np.NaN, inplace=True)
노원행정동1['pm10_y'].replace([-999,-9999], np.NaN, inplace=True)

노원행정동1['pm25_x'].replace([-999,-9999], np.NaN, inplace=True)   
노원행정동1['pm25_y'].replace([-999,-9999], np.NaN, inplace=True)   


# 시간데이터 분, 초만 잘라내서 'temp'라는 이름의 변수로 넣어줌
temp = [str(i)[:-6] for i in 노원행정동1.tm]
노원행정동1['temp'] = temp


# 미세먼지
# 같은 동 내 다른 스테이션에서 측정된 미세먼지농도를 하나의 리스트로 묶은 것 'mise'라는 이름의 변수를 가지는 데이터프레임 생성
tlist = []
for i in 노원행정동1[['temp','pm10_x', 'pm10_y']].values:  
    tlist.append([i[0],[i[1], i[2]]])

    
tdataframe = pd.DataFrame(tlist, columns = ['temp', 'mise'])
tdataframe.head()
no1 = pd.DataFrame(tdataframe.groupby('temp')['mise'].sum())


# 미세먼지 중앙값 구하기

mise_med = []

for i in no1.values:
    mise_med.append(s.median(i[0]))
no1['mise_med'] = mise_med


# 미세먼지 최빈값 구하기 
# 위와 같은 일을 하는데 1의자리에서 버린 값을 넣어줌 (최빈구간 구하기 위해)

gugan = []
for i in 노원행정동1[['temp', 'pm10_x', 'pm10_y']].values:
     gugan.append([i[0],[(i[1]//10)*10, (i[2]//10)*10]])


gugan_1 = pd.DataFrame(gugan, columns = ['temp', 'gugan']).groupby('temp')['gugan'].sum()
gugan_1.head()


# 전체 데이터에 적용
mise_mode = []
for i in range(len(gugan_1)):
  mise_mode.append(mode(gugan_1.values[i]))


# 초미세먼지
# 같은 동 내 다른 스테이션에서 측정된 미세먼지농도를 하나의 리스트로 묶은 것 'mise'라는 이름의 변수를 가지는 데이터프레임 생성

tlist = []
for i in 노원행정동1[['temp','pm25_x', 'pm25_y']].values:   
    tlist.append([i[0],[i[1], i[2]]])

tdataframe = pd.DataFrame(tlist, columns = ['temp', 'c_mise'])
tdataframe.head()
no1_1 = pd.DataFrame(tdataframe.groupby('temp')['c_mise'].sum())


# 초미세먼지 중앙값 구하기

mise_med = []

for i in no1_1.values:
    mise_med.append(s.median(i[0]))
no1_1['c_mise_med'] = mise_med


# 초미세먼지 최빈값 구하기
# 위와 같은 일을 하는데 1의자리에서 버린 값을 넣어줌 (최빈구간 구하기 위해)

gugan = []
for i in 노원행정동1[['temp','pm25_x', 'pm25_y']].values:
    gugan.append([i[0],[(i[1]//10)*10, (i[2]//10)*10]])
    
gugan_1 = pd.DataFrame(gugan, columns = ['temp', 'gugan']).groupby('temp')['gugan'].sum()
gugan_1


# 전체 데이터에 적용
cmise_mode = []
for i in range(len(gugan_1)):
  cmise_mode.append(mode(gugan_1.values[i]))

# 하나의 데이터프레임으로 만들기

no1.reset_index(inplace=True)
no1_1.reset_index(inplace=True)

all_no1 = pd.merge(no1[['temp', 'mise_med']], no1_1[['temp','c_mise_med']], on = 'temp')

all_no1.columns = [['공릉1동','공릉1동','공릉1동'],['temp','mise_med','c_mise_med']]

all_no1['mise_mode'] = pd.DataFrame(mise_mode)

all_no1['c_mise_mode'] = pd.DataFrame(cmise_mode)

all_no1.columns = ['temp','mise_med','c_mise_med','mise_mode','c_mise_mode']

all_no1['행정동명'] = '공릉1동'

all_no1.head()


## 2. 공릉2동(station 2개)

In [0]:
n_hdong.loc[no_dong[1]]

In [0]:
# 행정동 - nowon2(공릉2동) - station 2개
# 행정동별 datetime변경 및 이상치 처리 

노원행정동2 = multi_st(1)

# tm 변수를 시간 변수로 바꾸고 인덱스로 설정해주기
노원행정동2['tm'] = pd.to_datetime(노원행정동2['tm'], format = '%Y%m%d%H%M')

#하고 싶은 것 :   
#동별 시간대별 미세먼지 농도
# -999, -9999 na로 바꾸고 해보자

노원행정동2['pm10_x'].replace([-999,-9999], np.NaN, inplace=True)
노원행정동2['pm10_y'].replace([-999,-9999], np.NaN, inplace=True)

노원행정동2['pm25_x'].replace([-999,-9999], np.NaN, inplace=True)   
노원행정동2['pm25_y'].replace([-999,-9999], np.NaN, inplace=True)   


# 시간데이터 분, 초만 잘라내서 'temp'라는 이름의 변수로 넣어줌
temp = [str(i)[:-6] for i in 노원행정동2.tm]
노원행정동2['temp'] = temp


# 미세먼지
# 같은 동 내 다른 스테이션에서 측정된 미세먼지농도를 하나의 리스트로 묶은 것 'mise'라는 이름의 변수를 가지는 데이터프레임 생성

tlist = []
for i in 노원행정동2[['temp','pm10_x', 'pm10_y']].values:   
    tlist.append([i[0],[i[1], i[2]]])

    
tdataframe = pd.DataFrame(tlist, columns = ['temp', 'mise'])
tdataframe.head()
no2 = pd.DataFrame(tdataframe.groupby('temp')['mise'].sum())


# 미세먼지 중앙값 구하기

mise_med = []

for i in no2.values:
    mise_med.append(s.median(i[0]))
no2['mise_med'] = mise_med


# 미세먼지 최빈값 구하기 
# 위와 같은 일을 하는데 1의자리에서 버린 값을 넣어줌 (최빈구간 구하기 위해)

gugan = []
for i in 노원행정동2[['temp', 'pm10_x', 'pm10_y']].values:
     gugan.append([i[0],[(i[1]//10)*10, (i[2]//10)*10]])


gugan_2 = pd.DataFrame(gugan, columns = ['temp', 'gugan']).groupby('temp')['gugan'].sum()
gugan_2.head()


# 전체 데이터에 적용
mise_mode = []
for i in range(len(gugan_2)):
  mise_mode.append(mode(gugan_2.values[i]))


# 초미세먼지
# 같은 동 내 다른 스테이션에서 측정된 미세먼지농도를 하나의 리스트로 묶은 것 'mise'라는 이름의 변수를 가지는 데이터프레임 생성

tlist = []
for i in 노원행정동2[['temp','pm25_x', 'pm25_y']].values:  
    tlist.append([i[0],[i[1], i[2]]])

tdataframe = pd.DataFrame(tlist, columns = ['temp', 'c_mise'])
tdataframe.head()
no2_1 = pd.DataFrame(tdataframe.groupby('temp')['c_mise'].sum())

# 초미세먼지 중앙값 구하기

mise_med = []

for i in no2_1.values:
    mise_med.append(s.median(i[0]))
no2_1['c_mise_med'] = mise_med


# 초미세먼지 최빈값 구하기
# 위와 같은 일을 하는데 1의자리에서 버린 값을 넣어줌 (최빈구간 구하기 위해)

gugan = []
for i in 노원행정동2[['temp','pm25_x', 'pm25_y']].values:
    gugan.append([i[0],[(i[1]//10)*10, (i[2]//10)*10]])
    
gugan_2 = pd.DataFrame(gugan, columns = ['temp', 'gugan']).groupby('temp')['gugan'].sum()
gugan_2

# 전체 데이터에 적용
cmise_mode = []
for i in range(len(gugan_2)):
  cmise_mode.append(mode(gugan_2.values[i]))


# 하나의 데이터프레임으로 만들기

no2.reset_index(inplace=True)
no2_1.reset_index(inplace=True)

all_no2 = pd.merge(no1[['temp', 'mise_med']], no1_1[['temp','c_mise_med']], on = 'temp')

all_no2.columns = [['공릉2동','공릉2동','공릉2동'],['temp','mise_med','c_mise_med']]

all_no2['mise_mode'] = pd.DataFrame(mise_mode)

all_no2['c_mise_mode'] = pd.DataFrame(cmise_mode)

all_no2.columns = ['temp','mise_med','c_mise_med','mise_mode','c_mise_mode']

all_no2['행정동명'] = '공릉2동'

all_no2.head()


## 3. 상계10동(station 1개)

In [0]:
# 행정동 - nowon3(상계10동) - station 1개
filepath = 'C:\Python\JupyterNotebook\[공모전]빅콘\DATASET(미세먼지)\환경기상데이터\노원구\{}.csv'\
.format(n_hdong.loc[no_dong[2]]['스테이션.1'])
노원행정동3 = pd.read_csv(filepath)
노원행정동3.drop(['Unnamed: 10','co2','vocs', 'flag', 'noise'],axis=1, inplace=True)
노원행정동3['DONG_NM'] = no_dong[2]
노원행정동3.head()

In [0]:
# 행정동 - nowon3(상계10동) - station 1개
# 행정동별 datetime변경 및 이상치 처리 


노원행정동3['tm'] = pd.to_datetime(노원행정동3['tm'], format = '%Y%m%d%H%M')


#하고 싶은 것 :   
#동별 시간대별 미세먼지 농도
# -999, -9999 na로 바꾸고 해보자

노원행정동3['pm10'].replace([-999,-9999], np.NaN, inplace=True)

노원행정동3['pm25'].replace([-999,-9999], np.NaN, inplace=True)   

노원행정동3.describe(include='all')

# 시간데이터 분, 초만 잘라내서 'temp'라는 이름의 변수로 넣어줌
temp = [str(i)[:-6] for i in 노원행정동3.tm]
노원행정동3['temp'] = temp


# 미세먼지
# 같은 동 내 다른 스테이션에서 측정된 미세먼지농도를 하나의 리스트로 묶은 것 'mise'라는 이름의 변수를 가지는 데이터프레임 생성

tlist = []
for i in 노원행정동3[['temp','pm10']].values:   # 미세먼지 데이터에서 시간(에서 분,초 뺀 것), 미세먼지만 떼어 냄
    tlist.append([i[0],i[1]])

노원행정동3_미세먼지 = pd.DataFrame(tlist, columns = ['temp', 'mise_med'])
노원행정동3_미세먼지['mise_med'].replace([-999,-9999], np.NaN, inplace=True)

# 미세먼지 중앙값 구하기

노원행정동3_median = 노원행정동3_미세먼지.pivot_table(index='temp',
                           aggfunc='median', 
                           values='mise_med')

no3 = pd.DataFrame(노원행정동3_median)


# 미세먼지 최빈값 구하기 
# 위와 같은 일을 하는데 1의자리에서 버린 값을 넣어줌 (최빈구간 구하기 위해)

gugan = []
for i in 노원행정동3[['temp', 'pm10']].values:
     gugan.append([i[0],[(i[1]//10)*10]])


gugan_3 = pd.DataFrame(gugan, columns = ['temp', 'gugan']).groupby('temp')['gugan'].sum()
gugan_3.head()



# 전체 데이터에 적용
mise_mode = []
for i in range(len(gugan_3)):
  mise_mode.append(mode(gugan_3.values[i]))


# 초미세먼지
# 같은 동 내 다른 스테이션에서 측정된 미세먼지농도를 하나의 리스트로 묶은 것 'mise'라는 이름의 변수를 가지는 데이터프레임 생성

tlist = []
for i in 노원행정동3[['temp','pm25']].values:   # 미세먼지 데이터에서 시간(에서 분,초 뺀 것), 미세먼지만 떼어 냄
    tlist.append([i[0],i[1]])


    
노원행정동3_초미세먼지 = pd.DataFrame(tlist, columns = ['temp', 'c_mise_med'])
노원행정동3_초미세먼지['c_mise_med'].replace([-999,-9999], np.NaN, inplace=True)


# 초미세먼지 중앙값 구하기    

노원행정동3_Cmedian = 노원행정동3_초미세먼지.pivot_table(index='temp',
                           aggfunc='median', 
                           values='c_mise_med')


no3_1 = pd.DataFrame(노원행정동3_Cmedian)


# 초미세먼지 최빈값 구하기 

gugan = []
for i in 노원행정동3[['temp', 'pm25']].values:
     gugan.append([i[0],[(i[1]//10)*10]])


gugan_3 = pd.DataFrame(gugan, columns = ['temp', 'gugan']).groupby('temp')['gugan'].sum()
gugan_3.head()



# 전체 데이터에 적용
cmise_mode = []
for i in range(len(gugan_3)):
  cmise_mode.append(mode(gugan_3.values[i]))


# 하나의 데이터프레임으로 만들기

no3.reset_index(inplace=True)
no3_1.reset_index(inplace=True)

all_no3 = pd.merge(no3[['temp', 'mise_med']], no3_1[['temp','c_mise_med']], on = 'temp')

all_no3.columns = [['상계10동','상계10동','상계10동'],['temp','mise_med','c_mise_med']]

all_no3['mise_mode'] = pd.DataFrame(mise_mode)

all_no3['c_mise_mode'] = pd.DataFrame(cmise_mode)

all_no3.columns = ['temp','mise_med','c_mise_med','mise_mode','c_mise_mode']

all_no3['행정동명'] = '상계10동'

all_no3.head()



## 4. 상계1동(station 1개)

In [0]:
# 행정동 - nowon4(상계1동) - station 1개
filepath = 'C:\Python\JupyterNotebook\[공모전]빅콘\DATASET(미세먼지)\환경기상데이터\노원구\{}.csv'\
.format(n_hdong.loc[no_dong[3]]['스테이션.1'])
노원행정동4 = pd.read_csv(filepath)
노원행정동4.drop(['Unnamed: 10','co2','vocs', 'flag', 'noise'],axis=1, inplace=True)
노원행정동4['DONG_NM'] = no_dong[3]
노원행정동4.head()

In [0]:
# 행정동 - nowon4(상계1동) - station 1개
# 행정동별 datetime변경 및 이상치 처리 

# tm 변수를 시간 변수로 바꾸고 인덱스로 설정해주기
노원행정동4['tm'] = pd.to_datetime(노원행정동4['tm'], format = '%Y%m%d%H%M')

#하고 싶은 것 :   
#동별 시간대별 미세먼지 농도
# -999, -9999 na로 바꾸고 해보자

노원행정동4['pm10'].replace([-999,-9999], np.NaN, inplace=True)

노원행정동4['pm25'].replace([-999,-9999], np.NaN, inplace=True)   

노원행정동4.describe(include='all')

# 시간데이터 분, 초만 잘라내서 'temp'라는 이름의 변수로 넣어줌

temp = [str(i)[:-6] for i in 노원행정동4.tm]
노원행정동4['temp'] = temp


# 미세먼지
# 같은 동 내 다른 스테이션에서 측정된 미세먼지농도를 하나의 리스트로 묶은 것 'mise'라는 이름의 변수를 가지는 데이터프레임 생성

tlist = []
for i in 노원행정동4[['temp','pm10']].values:   # 미세먼지 데이터에서 시간(에서 분,초 뺀 것), 미세먼지만 떼어 냄
    tlist.append([i[0],i[1]])

노원행정동4_미세먼지 = pd.DataFrame(tlist, columns = ['temp', 'mise_med'])
노원행정동4_미세먼지.head()

# 미세먼지 중앙값 구하기

노원행정동4_median = 노원행정동4_미세먼지.pivot_table(index='temp',
                           aggfunc='median', 
                           values='mise_med')

no4 = pd.DataFrame(노원행정동4_median)


# 미세먼지 최빈값 구하기 
# 위와 같은 일을 하는데 1의자리에서 버린 값을 넣어줌 (최빈구간 구하기 위해)

gugan = []
for i in 노원행정동4[['temp', 'pm10']].values:
     gugan.append([i[0],[(i[1]//10)*10]])


gugan_4 = pd.DataFrame(gugan, columns = ['temp', 'gugan']).groupby('temp')['gugan'].sum()
gugan_4.head()


# 전체 데이터에 적용
mise_mode = []
for i in range(len(gugan_4)):
  mise_mode.append(mode(gugan_4.values[i]))



# 초미세먼지
# 같은 동 내 다른 스테이션에서 측정된 미세먼지농도를 하나의 리스트로 묶은 것 'mise'라는 이름의 변수를 가지는 데이터프레임 생성

tlist = []
for i in 노원행정동4[['temp','pm25']].values:   # 초미세먼지 데이터에서 시간(에서 분,초 뺀 것), 미세먼지만 떼어 냄
    tlist.append([i[0],i[1]])

노원행정동4_초미세먼지 = pd.DataFrame(tlist, columns = ['temp', 'c_mise_med'])
노원행정동4_초미세먼지['c_mise_med'].replace([-999,-9999], np.NaN, inplace=True)


# 초미세먼지 중앙값 구하기 

노원행정동4_Cmedian = 노원행정동4_초미세먼지.pivot_table(index='temp',
                           aggfunc='median', 
                           values='c_mise_med')


no4_1 = pd.DataFrame(노원행정동4_Cmedian)

gugan = []
for i in 노원행정동4[['temp', 'pm25']].values:
     gugan.append([i[0],[(i[1]//10)*10]])


gugan_4 = pd.DataFrame(gugan, columns = ['temp', 'gugan']).groupby('temp')['gugan'].sum()
gugan_4.head()


# 전체 데이터에 적용
cmise_mode = []
for i in range(len(gugan_4)):
  cmise_mode.append(mode(gugan_4.values[i]))


# 하나의 데이터프레임으로 만들기
no4.reset_index(inplace=True)
no4_1.reset_index(inplace=True)

all_no4 = pd.merge(no4[['temp', 'mise_med']], no4_1[['temp','c_mise_med']], on = 'temp')

all_no4.columns = [['상계1동','상계1동','상계1동'],['temp','mise_med','c_mise_med']]

all_no4['mise_mode'] = pd.DataFrame(mise_mode)

all_no4['c_mise_mode'] = pd.DataFrame(cmise_mode)

all_no4.columns = ['temp','mise_med','c_mise_med','mise_mode','c_mise_mode']

all_no4['행정동명'] = '상계1동'

all_no4.head()



## 5. 상계2동(station 4개)

In [0]:
# 행정동 - nowon5(상계2동) - station 4개
노원행정동5 = multi_st(4)
노원행정동5.head()


In [0]:
# 행정동 - nowon5(상계2동) - station 4개
# 행정동별 datetime변경 및 이상치 처리 

# tm 변수를 시간 변수로 바꾸고 인덱스로 설정해주기
노원행정동5['tm'] = pd.to_datetime(노원행정동5['tm'], format = '%Y%m%d%H%M')

#하고 싶은 것 :   
#동별 시간대별 미세먼지 농도
# -999, -9999 na로 바꾸고 해보자

노원행정동5['pm10_x'].replace([-999,-9999], np.NaN, inplace=True)
노원행정동5['pm10_y'].replace([-999,-9999], np.NaN, inplace=True)

노원행정동5['pm25_x'].replace([-999,-9999], np.NaN, inplace=True)   
노원행정동5['pm25_y'].replace([-999,-9999], np.NaN, inplace=True)   

노원행정동5.describe(include='all')

# 시간데이터 분, 초만 잘라내서 'temp'라는 이름의 변수로 넣어줌
temp = [str(i)[:-6] for i in 노원행정동5.tm]
노원행정동5['temp'] = temp


# 미세먼지
# 같은 동 내 다른 스테이션에서 측정된 미세먼지농도를 하나의 리스트로 묶은 것 'mise'라는 이름의 변수를 가지는 데이터프레임 생성

tlist = []
for i in 노원행정동5[['temp','pm10_x', 'pm10_y']].values:   # 미세먼지 데이터에서 시간(에서 분,초 뺀 것), 미세먼지만 떼어 냄
    tlist.append([i[0],[i[1], i[2]]])

    
tdataframe = pd.DataFrame(tlist, columns = ['temp', 'mise'])
tdataframe.head()
no5 = pd.DataFrame(tdataframe.groupby('temp')['mise'].sum())


# 미세먼지 중앙값 구하기

mise_med = []

for i in no5.values:
    mise_med.append(s.median(i[0]))
no5['mise_med'] = mise_med


# 미세먼지 최빈값 구하기 
# 위와 같은 일을 하는데 1의자리에서 버린 값을 넣어줌 (최빈구간 구하기 위해)

gugan = []
for i in 노원행정동5[['temp', 'pm10_x', 'pm10_y']].values:
     gugan.append([i[0],[(i[1]//10)*10, (i[2]//10)*10]])


gugan_5 = pd.DataFrame(gugan, columns = ['temp', 'gugan']).groupby('temp')['gugan'].sum()
gugan_5.head()


# 전체 데이터에 적용
mise_mode = []
for i in range(len(gugan_5)):
  mise_mode.append(mode(gugan_5.values[i]))


# 초미세먼지
# 같은 동 내 다른 스테이션에서 측정된 미세먼지농도를 하나의 리스트로 묶은 것 'mise'라는 이름의 변수를 가지는 데이터프레임 생성

tlist = []
for i in 노원행정동5[['temp','pm25_x', 'pm25_y']].values:   # 초미세먼지 데이터에서 시간(에서 분,초 뺀 것), 미세먼지만 떼어 냄
    tlist.append([i[0],[i[1], i[2]]])

tdataframe = pd.DataFrame(tlist, columns = ['temp', 'c_mise'])
tdataframe.head()
no5_1 = pd.DataFrame(tdataframe.groupby('temp')['c_mise'].sum())

# 초미세먼지 중앙값 구하기

mise_med = []

for i in no5_1.values:
    mise_med.append(s.median(i[0]))
no5_1['c_mise_med'] = mise_med


# 초미세먼지 최빈값 구하기
# 위와 같은 일을 하는데 1의자리에서 버린 값을 넣어줌 (최빈구간 구하기 위해)

gugan = []
for i in 노원행정동5[['temp','pm25_x', 'pm25_y']].values:
    gugan.append([i[0],[(i[1]//10)*10, (i[2]//10)*10]])
    
gugan_5 = pd.DataFrame(gugan, columns = ['temp', 'gugan']).groupby('temp')['gugan'].sum()
gugan_5


# 전체 데이터에 적용
cmise_mode = []
for i in range(len(gugan_5)):
  cmise_mode.append(mode(gugan_5.values[i]))


# 하나의 데이터프레임으로 만들기
no5.reset_index(inplace=True)
no5_1.reset_index(inplace=True)

all_no5 = pd.merge(no5[['temp', 'mise_med']], no5_1[['temp','c_mise_med']], on = 'temp')

all_no5.columns = [['상계2동','상계2동','상계2동'],['temp','mise_med','c_mise_med']]

all_no5['mise_mode'] = pd.DataFrame(mise_mode)

all_no5['c_mise_mode'] = pd.DataFrame(cmise_mode)

all_no5.columns = ['temp','mise_med','c_mise_med','mise_mode','c_mise_mode']

all_no5['행정동명'] = '상계2동'

all_no5.head()


## 6. 상계3,4동(station 1개)

In [0]:
# 행정동 - nowon6(상계3,4동) - station 1개
filepath = 'C:\Python\JupyterNotebook\[공모전]빅콘\DATASET(미세먼지)\환경기상데이터\노원구\{}.csv'\
.format(n_hdong.loc[no_dong[5]]['스테이션.1'])
노원행정동6 = pd.read_csv(filepath)
노원행정동6.drop(['Unnamed: 10','co2','vocs', 'flag', 'noise'],axis=1, inplace=True)
노원행정동6['DONG_NM'] = no_dong[5]
노원행정동6.head()

In [0]:
# 행정동 - nowon4(상계1동) - station 1개
# 행정동별 datetime변경 및 이상치 처리 


# tm 변수를 시간 변수로 바꾸고 인덱스로 설정해주기
노원행정동6['tm'] = pd.to_datetime(노원행정동6['tm'], format = '%Y%m%d%H%M')

#하고 싶은 것 :   
#동별 시간대별 미세먼지 농도
# -999, -9999 na로 바꾸고 해보자

노원행정동6['pm10'].replace([-999,-9999], np.NaN, inplace=True)

노원행정동6['pm25'].replace([-999,-9999], np.NaN, inplace=True)   

노원행정동6.describe(include='all')

# 시간데이터 분, 초만 잘라내서 'temp'라는 이름의 변수로 넣어줌

temp = [str(i)[:-6] for i in 노원행정동6.tm]
노원행정동6['temp'] = temp


# 미세먼지
# 같은 동 내 다른 스테이션에서 측정된 미세먼지농도를 하나의 리스트로 묶은 것 'mise'라는 이름의 변수를 가지는 데이터프레임 생성

tlist = []
for i in 노원행정동6[['temp','pm10']].values:   # 미세먼지 데이터에서 시간(에서 분,초 뺀 것), 미세먼지만 떼어 냄
    tlist.append([i[0],i[1]])

노원행정동6_미세먼지 = pd.DataFrame(tlist, columns = ['temp', 'mise_med'])
노원행정동6_미세먼지.head()


# 미세먼지 중앙값 구하기
노원행정동6_median = 노원행정동6_미세먼지.pivot_table(index='temp',
                           aggfunc='median', 
                           values='mise_med')

no6 = pd.DataFrame(노원행정동6_median)

# 미세먼지 최빈값 구하기 
# 위와 같은 일을 하는데 1의자리에서 버린 값을 넣어줌 (최빈구간 구하기 위해)

gugan = []
for i in 노원행정동6[['temp', 'pm10']].values:
     gugan.append([i[0],[(i[1]//10)*10]])


gugan_6 = pd.DataFrame(gugan, columns = ['temp', 'gugan']).groupby('temp')['gugan'].sum()
gugan_6.head()

# 전체 데이터에 적용
mise_mode = []
for i in range(len(gugan_6)):
  mise_mode.append(mode(gugan_6.values[i]))



# 초미세먼지
# 같은 동 내 다른 스테이션에서 측정된 미세먼지농도를 하나의 리스트로 묶은 것 'mise'라는 이름의 변수를 가지는 데이터프레임 생성

tlist = []
for i in 노원행정동6[['temp','pm25']].values:   # 초미세먼지 데이터에서 시간(에서 분,초 뺀 것), 미세먼지만 떼어 냄
    tlist.append([i[0],i[1]])

노원행정동6_초미세먼지 = pd.DataFrame(tlist, columns = ['temp', 'c_mise_med'])
노원행정동6_초미세먼지['c_mise_med'].replace([-999,-9999], np.NaN, inplace=True)


# 초미세먼지 중앙값 구하기 
노원행정동6_Cmedian = 노원행정동6_초미세먼지.pivot_table(index='temp',
                           aggfunc='median', 
                           values='c_mise_med')


no6_1 = pd.DataFrame(노원행정동6_Cmedian)


# 초미세먼지 최빈값 구하기 
gugan = []
for i in 노원행정동6[['temp', 'pm25']].values:
     gugan.append([i[0],[(i[1]//10)*10]])


gugan_6 = pd.DataFrame(gugan, columns = ['temp', 'gugan']).groupby('temp')['gugan'].sum()
gugan_6.head()


# 전체 데이터에 적용
cmise_mode = []
for i in range(len(gugan_6)):
  cmise_mode.append(mode(gugan_6.values[i]))


# 하나의 데이터프레임으로 만들기

no6.reset_index(inplace=True)
no6_1.reset_index(inplace=True)

all_no6 = pd.merge(no6[['temp', 'mise_med']], no6_1[['temp','c_mise_med']], on = 'temp')

all_no6.columns = [['상계3.4동','상계3.4동','상계3.4동'],['temp','mise_med','c_mise_med']]

all_no6['mise_mode'] = pd.DataFrame(mise_mode)

all_no6['c_mise_mode'] = pd.DataFrame(cmise_mode)

all_no6.columns = ['temp','mise_med','c_mise_med','mise_mode','c_mise_mode']

all_no6['행정동명'] = '상계3,4동'

all_no6.head()



## 7. 상계5동(station 2개)

In [0]:
# 행정동 - nowon7(상계5동) - station 2개
노원행정동7 = multi_st(6)
노원행정동7.head()    

In [0]:
# 행정동별 datetime변경 및 이상치 처리 
# 행정동 - nowon7(상계5동) - station 2개

노원행정동7 = multi_st(1)

# tm 변수를 시간 변수로 바꾸고 인덱스로 설정해주기

노원행정동7['tm'] = pd.to_datetime(노원행정동7['tm'], format = '%Y%m%d%H%M')

#하고 싶은 것 :   
#동별 시간대별 미세먼지 농도
# -999, -9999 na로 바꾸고 해보자

노원행정동7['pm10_x'].replace([-999,-9999], np.NaN, inplace=True)
노원행정동7['pm10_y'].replace([-999,-9999], np.NaN, inplace=True)

노원행정동7['pm25_x'].replace([-999,-9999], np.NaN, inplace=True)   
노원행정동7['pm25_y'].replace([-999,-9999], np.NaN, inplace=True)   

노원행정동7.describe(include='all')

# 시간데이터 분, 초만 잘라내서 'temp'라는 이름의 변수로 넣어줌

temp = [str(i)[:-6] for i in 노원행정동7.tm]
노원행정동7['temp'] = temp


# 미세먼지
# 같은 동 내 다른 스테이션에서 측정된 미세먼지농도를 하나의 리스트로 묶은 것 'mise'라는 이름의 변수를 가지는 데이터프레임 생성

tlist = []
for i in 노원행정동7[['temp','pm10_x', 'pm10_y']].values:   # 미세먼지 데이터에서 시간(에서 분,초 뺀 것), 미세먼지만 떼어 냄
    tlist.append([i[0],[i[1], i[2]]])

    
tdataframe = pd.DataFrame(tlist, columns = ['temp', 'mise'])
tdataframe.head()
no7 = pd.DataFrame(tdataframe.groupby('temp')['mise'].sum())


# 미세먼지 중앙값 구하기

mise_med = []

for i in no7.values:
    mise_med.append(s.median(i[0]))
no7['mise_med'] = mise_med


# 미세먼지 최빈값 구하기 
# 위와 같은 일을 하는데 1의자리에서 버린 값을 넣어줌 (최빈구간 구하기 위해)

gugan = []
for i in 노원행정동7[['temp', 'pm10_x', 'pm10_y']].values:
     gugan.append([i[0],[(i[1]//10)*10, (i[2]//10)*10]])


gugan_7 = pd.DataFrame(gugan, columns = ['temp', 'gugan']).groupby('temp')['gugan'].sum()
gugan_7.head()


# 전체 데이터에 적용
mise_mode = []
for i in range(len(gugan_7)):
  mise_mode.append(mode(gugan_7.values[i]))


# 초미세먼지
# 같은 동 내 다른 스테이션에서 측정된 미세먼지농도를 하나의 리스트로 묶은 것 'mise'라는 이름의 변수를 가지는 데이터프레임 생성

tlist = []
for i in 노원행정동7[['temp','pm25_x', 'pm25_y']].values:   # 초미세먼지 데이터에서 시간(에서 분,초 뺀 것), 미세먼지만 떼어 냄
    tlist.append([i[0],[i[1], i[2]]])

tdataframe = pd.DataFrame(tlist, columns = ['temp', 'c_mise'])
tdataframe.head()
no7_1 = pd.DataFrame(tdataframe.groupby('temp')['c_mise'].sum())

# 초미세먼지 중앙값 구하기

mise_med = []

for i in no7_1.values:
    mise_med.append(s.median(i[0]))
no7_1['c_mise_med'] = mise_med


# 초미세먼지 최빈값 구하기
# 위와 같은 일을 하는데 1의자리에서 버린 값을 넣어줌 (최빈구간 구하기 위해)

gugan = []
for i in 노원행정동7[['temp','pm25_x', 'pm25_y']].values:
    gugan.append([i[0],[(i[1]//10)*10, (i[2]//10)*10]])
    
gugan_7 = pd.DataFrame(gugan, columns = ['temp', 'gugan']).groupby('temp')['gugan'].sum()
gugan_7

# 전체 데이터에 적용
cmise_mode = []
for i in range(len(gugan_7)):
  cmise_mode.append(mode(gugan_7.values[i]))


# 하나의 데이터프레임으로 만들기
no7.reset_index(inplace=True)
no7_1.reset_index(inplace=True)

all_no7 = pd.merge(no7[['temp', 'mise_med']], no7_1[['temp','c_mise_med']], on = 'temp')

all_no7.columns = [['상계5동','상계5동','상계5동'],['temp','mise_med','c_mise_med']]

all_no7['mise_mode'] = pd.DataFrame(mise_mode)

all_no7['c_mise_mode'] = pd.DataFrame(cmise_mode)

all_no7.columns = ['temp','mise_med','c_mise_med','mise_mode','c_mise_mode']

all_no7['행정동명'] = '상계5동'


all_no7.head()


## 8. 상계6,7동(station 3개)

In [0]:
# 행정동 - nowon8(상계6,7동) - station 3개
노원행정동8 = multi_st(7)
노원행정동8.head()    

In [0]:
노원행정동8.describe()

In [0]:
# tm 변수를 시간 변수로 바꾸고 인덱스로 설정해주기
노원행정동8['tm'] = pd.to_datetime(노원행정동8['tm'], format = '%Y%m%d%H%M')

#하고 싶은 것 :   
#동별 시간대별 미세먼지 농도
# -999, -9999 na로 바꾸고 해보자

노원행정동8['pm10_x'].replace([-999,-9999], np.NaN, inplace=True)
노원행정동8['pm10_y'].replace([-999,-9999], np.NaN, inplace=True)

노원행정동8['pm25_x'].replace([-999,-9999], np.NaN, inplace=True)   
노원행정동8['pm25_y'].replace([-999,-9999], np.NaN, inplace=True)   

노원행정동8.describe(include='all')

# 시간데이터 분, 초만 잘라내서 'temp'라는 이름의 변수로 넣어줌

temp = [str(i)[:-6] for i in 노원행정동8.tm]
노원행정동8['temp'] = temp


# 미세먼지
# 같은 동 내 다른 스테이션에서 측정된 미세먼지농도를 하나의 리스트로 묶은 것 'mise'라는 이름의 변수를 가지는 데이터프레임 생성

tlist = []
for i in 노원행정동8[['temp','pm10_x', 'pm10_y']].values:   # 미세먼지 데이터에서 시간(에서 분,초 뺀 것), 미세먼지만 떼어 냄
    tlist.append([i[0],[i[1], i[2]]])

    
tdataframe = pd.DataFrame(tlist, columns = ['temp', 'mise'])
tdataframe.head()
no8 = pd.DataFrame(tdataframe.groupby('temp')['mise'].sum())


# 미세먼지 중앙값 구하기

mise_med = []

for i in no8.values:
    mise_med.append(s.median(i[0]))
no8['mise_med'] = mise_med


# 미세먼지 최빈값 구하기 
# 위와 같은 일을 하는데 1의자리에서 버린 값을 넣어줌 (최빈구간 구하기 위해)

gugan = []
for i in 노원행정동8[['temp', 'pm10_x', 'pm10_y']].values:
     gugan.append([i[0],[(i[1]//10)*10, (i[2]//10)*10]])


gugan_8 = pd.DataFrame(gugan, columns = ['temp', 'gugan']).groupby('temp')['gugan'].sum()
gugan_8.head()


# 전체 데이터에 적용
mise_mode = []
for i in range(len(gugan_8)):
  mise_mode.append(mode(gugan_8.values[i]))


# 초미세먼지
# 같은 동 내 다른 스테이션에서 측정된 미세먼지농도를 하나의 리스트로 묶은 것 'mise'라는 이름의 변수를 가지는 데이터프레임 생성

tlist = []
for i in 노원행정동8[['temp','pm25_x', 'pm25_y']].values:   # 초미세먼지 데이터에서 시간(에서 분,초 뺀 것), 미세먼지만 떼어 냄
    tlist.append([i[0],[i[1], i[2]]])

tdataframe = pd.DataFrame(tlist, columns = ['temp', 'c_mise'])
tdataframe.head()
no8_1 = pd.DataFrame(tdataframe.groupby('temp')['c_mise'].sum())

# 초미세먼지 중앙값 구하기

mise_med = []

for i in no8_1.values:
    mise_med.append(s.median(i[0]))
no8_1['c_mise_med'] = mise_med


# 초미세먼지 최빈값 구하기
# 위와 같은 일을 하는데 1의자리에서 버린 값을 넣어줌 (최빈구간 구하기 위해)

gugan = []
for i in 노원행정동8[['temp','pm25_x', 'pm25_y']].values:
    gugan.append([i[0],[(i[1]//10)*10, (i[2]//10)*10]])
    
gugan_8 = pd.DataFrame(gugan, columns = ['temp', 'gugan']).groupby('temp')['gugan'].sum()
gugan_8

# 전체 데이터에 적용
cmise_mode = []
for i in range(len(gugan_8)):
  cmise_mode.append(mode(gugan_8.values[i]))



# 하나의 데이터프레임으로 만들기
no8.reset_index(inplace=True)
no8_1.reset_index(inplace=True)

all_no8 = pd.merge(no8[['temp', 'mise_med']], no8_1[['temp','c_mise_med']], on = 'temp')

all_no8.columns = [['상계6.7동','상계6.7동','상계6.7동'],['temp','mise_med','c_mise_med']]

all_no8['mise_mode'] = pd.DataFrame(mise_mode)

all_no8['c_mise_mode'] = pd.DataFrame(cmise_mode)

all_no8.columns = ['temp','mise_med','c_mise_med','mise_mode','c_mise_mode']

all_no8['행정동명'] = '상계6,7동'


all_no8.head()


## 9. 월계1동(station 1개)

In [0]:
# 행정동 - nowon9(월계1동) - station 1개
filepath = 'C:\Python\JupyterNotebook\[공모전]빅콘\DATASET(미세먼지)\환경기상데이터\노원구\{}.csv'\
.format(n_hdong.loc[no_dong[8]]['스테이션.1'])
노원행정동9 = pd.read_csv(filepath)
노원행정동9.drop(['Unnamed: 10','co2','vocs', 'flag', 'noise'],axis=1, inplace=True)
노원행정동9['DONG_NM'] = no_dong[8]
노원행정동9.head()

In [0]:
# 행정동 - nowon9(월계1동) - station 1개
# 행정동별 datetime변경 및 이상치 처리 

# tm 변수를 시간 변수로 바꾸고 인덱스로 설정해주기
노원행정동9['tm'] = pd.to_datetime(노원행정동9['tm'], format = '%Y%m%d%H%M')

#하고 싶은 것 :   
#동별 시간대별 미세먼지 농도
# -999, -9999 na로 바꾸고 해보자

노원행정동9['pm10'].replace([-999,-9999], np.NaN, inplace=True)

노원행정동9['pm25'].replace([-999,-9999], np.NaN, inplace=True)   

# 시간데이터 분, 초만 잘라내서 'temp'라는 이름의 변수로 넣어줌

temp = [str(i)[:-6] for i in 노원행정동9.tm]
노원행정동9['temp'] = temp


# 미세먼지
# 같은 동 내 다른 스테이션에서 측정된 미세먼지농도를 하나의 리스트로 묶은 것 'mise'라는 이름의 변수를 가지는 데이터프레임 생성

tlist = []
for i in 노원행정동9[['temp','pm10']].values:   # 미세먼지 데이터에서 시간(에서 분,초 뺀 것), 미세먼지만 떼어 냄
    tlist.append([i[0],i[1]])

노원행정동9_미세먼지 = pd.DataFrame(tlist, columns = ['temp', 'mise_med'])
노원행정동9_미세먼지.head()


# 미세먼지 중앙값 구하기
노원행정동9_median = 노원행정동9_미세먼지.pivot_table(index='temp',
                           aggfunc='median', 
                           values='mise_med')

no9 = pd.DataFrame(노원행정동9_median)


# 미세먼지 최빈값 구하기 
# 위와 같은 일을 하는데 1의자리에서 버린 값을 넣어줌 (최빈구간 구하기 위해)

gugan = []
for i in 노원행정동9[['temp', 'pm10']].values:
     gugan.append([i[0],[(i[1]//10)*10]])


gugan_9 = pd.DataFrame(gugan, columns = ['temp', 'gugan']).groupby('temp')['gugan'].sum()
gugan_9.head()

# 전체 데이터에 적용
mise_mode = []
for i in range(len(gugan_9)):
  mise_mode.append(mode(gugan_9.values[i]))



# 초미세먼지
# 같은 동 내 다른 스테이션에서 측정된 미세먼지농도를 하나의 리스트로 묶은 것 'mise'라는 이름의 변수를 가지는 데이터프레임 생성

tlist = []
for i in 노원행정동9[['temp','pm25']].values:   # 초미세먼지 데이터에서 시간(에서 분,초 뺀 것), 미세먼지만 떼어 냄
    tlist.append([i[0],i[1]])

노원행정동9_초미세먼지 = pd.DataFrame(tlist, columns = ['temp', 'c_mise_med'])
노원행정동9_초미세먼지['c_mise_med'].replace([-999,-9999], np.NaN, inplace=True)


# 초미세먼지 중앙값 구하기 
노원행정동9_Cmedian = 노원행정동9_초미세먼지.pivot_table(index='temp',
                           aggfunc='median', 
                           values='c_mise_med')


no9_1 = pd.DataFrame(노원행정동9_Cmedian)


# 초미세먼지 최빈값 구하기 
gugan = []
for i in 노원행정동9[['temp', 'pm25']].values:
     gugan.append([i[0],[(i[1]//10)*10]])


gugan_9 = pd.DataFrame(gugan, columns = ['temp', 'gugan']).groupby('temp')['gugan'].sum()
gugan_9.head()


# 전체 데이터에 적용
cmise_mode = []
for i in range(len(gugan_9)):
  cmise_mode.append(mode(gugan_9.values[i]))


# 하나의 데이터프레임으로 만들기
no9.reset_index(inplace=True)
no9_1.reset_index(inplace=True)

all_no9 = pd.merge(no9[['temp', 'mise_med']], no9_1[['temp','c_mise_med']], on = 'temp')

all_no9.columns = [['월계1동','월계1동','월계1동'],['temp','mise_med','c_mise_med']]

all_no9['mise_mode'] = pd.DataFrame(mise_mode)

all_no9['c_mise_mode'] = pd.DataFrame(cmise_mode)

all_no9.columns = ['temp','mise_med','c_mise_med','mise_mode','c_mise_mode']

all_no9['행정동명'] = '월계1동'

all_no9.head()



## 10. 월계3동(station 1개)

In [0]:
# 행정동 - nowon10(월계3동) - station 1개
filepath = 'C:\Python\JupyterNotebook\[공모전]빅콘\DATASET(미세먼지)\환경기상데이터\노원구\{}.csv'\
.format(n_hdong.loc[no_dong[9]]['스테이션.1'])
노원행정동10 = pd.read_csv(filepath)
노원행정동10.drop(['Unnamed: 10','co2','vocs', 'flag', 'noise'],axis=1, inplace=True)
노원행정동10['DONG_NM'] = no_dong[9]
노원행정동10.head()

In [0]:
노원행정동10.describe()

In [0]:
# 행정동별 datetime변경 및 이상치 처리 
# 행정동 - nowon10(월계3동) - station 1개

# tm 변수를 시간 변수로 바꾸고 인덱스로 설정해주기
노원행정동10['tm'] = pd.to_datetime(노원행정동10['tm'], format = '%Y%m%d%H%M')

#하고 싶은 것 :   
#동별 시간대별 미세먼지 농도
# -999, -9999 na로 바꾸고 해보자

노원행정동10['pm10'].replace([-999,-9999], np.NaN, inplace=True)

노원행정동10['pm25'].replace([-999,-9999], np.NaN, inplace=True)   

노원행정동10.describe(include='all')

# 시간데이터 분, 초만 잘라내서 'temp'라는 이름의 변수로 넣어줌

temp = [str(i)[:-6] for i in 노원행정동10.tm]
노원행정동10['temp'] = temp


# 미세먼지
# 같은 동 내 다른 스테이션에서 측정된 미세먼지농도를 하나의 리스트로 묶은 것 'mise'라는 이름의 변수를 가지는 데이터프레임 생성

tlist = []
for i in 노원행정동10[['temp','pm10']].values:   # 미세먼지 데이터에서 시간(에서 분,초 뺀 것), 미세먼지만 떼어 냄
    tlist.append([i[0],i[1]])

노원행정동10_미세먼지 = pd.DataFrame(tlist, columns = ['temp', 'mise_med'])
노원행정동10_미세먼지.head()

# 미세먼지 중앙값 구하기
노원행정동10_median = 노원행정동10_미세먼지.pivot_table(index='temp',
                           aggfunc='median', 
                           values='mise_med')

no10 = pd.DataFrame(노원행정동10_median)

# 미세먼지 최빈값 구하기 
# 위와 같은 일을 하는데 1의자리에서 버린 값을 넣어줌 (최빈구간 구하기 위해)
gugan = []
for i in 노원행정동10[['temp', 'pm10']].values:
     gugan.append([i[0],[(i[1]//10)*10]])


gugan_10 = pd.DataFrame(gugan, columns = ['temp', 'gugan']).groupby('temp')['gugan'].sum()
gugan_10.head()

# 전체 데이터에 적용
mise_mode = []
for i in range(len(gugan_10)):
  mise_mode.append(mode(gugan_10.values[i]))



# 초미세먼지
# 같은 동 내 다른 스테이션에서 측정된 미세먼지농도를 하나의 리스트로 묶은 것 'mise'라는 이름의 변수를 가지는 데이터프레임 생성

tlist = []
for i in 노원행정동10[['temp','pm25']].values:   # 초미세먼지 데이터에서 시간(에서 분,초 뺀 것), 미세먼지만 떼어 냄
    tlist.append([i[0],i[1]])

노원행정동10_초미세먼지 = pd.DataFrame(tlist, columns = ['temp', 'c_mise_med'])
노원행정동10_초미세먼지['c_mise_med'].replace([-999,-9999], np.NaN, inplace=True)


# 초미세먼지 중앙값 구하기   
노원행정동10_Cmedian = 노원행정동10_초미세먼지.pivot_table(index='temp',
                           aggfunc='median', 
                           values='c_mise_med')


no10_1 = pd.DataFrame(노원행정동10_Cmedian)


# 초미세먼지 최빈값 구하기 
gugan = []
for i in 노원행정동10[['temp', 'pm25']].values:
     gugan.append([i[0],[(i[1]//10)*10]])


gugan_10 = pd.DataFrame(gugan, columns = ['temp', 'gugan']).groupby('temp')['gugan'].sum()
gugan_10.head()


# 전체 데이터에 적용
cmise_mode = []
for i in range(len(gugan_10)):
  cmise_mode.append(mode(gugan_10.values[i]))


# 하나의 데이터프레임으로 만들기
no10.reset_index(inplace=True)
no10_1.reset_index(inplace=True)

all_no10 = pd.merge(no9[['temp', 'mise_med']], no9_1[['temp','c_mise_med']], on = 'temp')

all_no10.columns = [['월계3동','월계3동','월계3동'],['temp','mise_med','c_mise_med']]

all_no10['mise_mode'] = pd.DataFrame(mise_mode)

all_no10['c_mise_mode'] = pd.DataFrame(cmise_mode)

all_no10.columns = ['temp','mise_med','c_mise_med','mise_mode','c_mise_mode']

all_no10['행정동명'] = '월계3동'

all_no10.head()



## 11. 중계 2,3동(station 1개)

In [0]:
# 행정동 - nowon11(중계2,3동) - station 1개
filepath = 'C:\Python\JupyterNotebook\[공모전]빅콘\DATASET(미세먼지)\환경기상데이터\노원구\{}.csv'\
.format(n_hdong.loc[no_dong[10]]['스테이션.1'])
노원행정동11 = pd.read_csv(filepath)
노원행정동11.drop(['Unnamed: 10','co2','vocs', 'flag', 'noise'],axis=1, inplace=True)
노원행정동11['DONG_NM'] = no_dong[10]
노원행정동11.head()

In [0]:
# 행정동별 datetime변경 및 이상치 처리 
# 행정동 - nowon11(중계2,3동) - station 1개

# tm 변수를 시간 변수로 바꾸고 인덱스로 설정해주기
노원행정동11['tm'] = pd.to_datetime(노원행정동11['tm'], format = '%Y%m%d%H%M')

#하고 싶은 것 :   
#동별 시간대별 미세먼지 농도
# -999, -9999 na로 바꾸고 해보자

노원행정동11['pm10'].replace([-999,-9999], np.NaN, inplace=True)

노원행정동11['pm25'].replace([-999,-9999], np.NaN, inplace=True)   

노원행정동11.describe(include='all')

# 시간데이터 분, 초만 잘라내서 'temp'라는 이름의 변수로 넣어줌

temp = [str(i)[:-6] for i in 노원행정동11.tm]
노원행정동11['temp'] = temp


# 미세먼지
# 같은 동 내 다른 스테이션에서 측정된 미세먼지농도를 하나의 리스트로 묶은 것 'mise'라는 이름의 변수를 가지는 데이터프레임 생성

tlist = []
for i in 노원행정동11[['temp','pm10']].values:   # 미세먼지 데이터에서 시간(에서 분,초 뺀 것), 미세먼지만 떼어 냄
    tlist.append([i[0],i[1]])

노원행정동11_미세먼지 = pd.DataFrame(tlist, columns = ['temp', 'mise_med'])
노원행정동11_미세먼지.head()

# 미세먼지 중앙값 구하기
노원행정동11_median = 노원행정동11_미세먼지.pivot_table(index='temp',
                           aggfunc='median', 
                           values='mise_med')

no11 = pd.DataFrame(노원행정동11_median)


# 미세먼지 최빈값 구하기
# 위와 같은 일을 하는데 1의자리에서 버린 값을 넣어줌 (최빈구간 구하기 위해)

gugan = []
for i in 노원행정동11[['temp', 'pm10']].values:
     gugan.append([i[0],[(i[1]//10)*10]])


gugan_11 = pd.DataFrame(gugan, columns = ['temp', 'gugan']).groupby('temp')['gugan'].sum()
gugan_11.head()

# 전체 데이터에 적용
mise_mode = []
for i in range(len(gugan_11)):
  mise_mode.append(mode(gugan_11.values[i]))



# 초미세먼지
# 같은 동 내 다른 스테이션에서 측정된 미세먼지농도를 하나의 리스트로 묶은 것 'mise'라는 이름의 변수를 가지는 데이터프레임 생성

tlist = []
for i in 노원행정동11[['temp','pm25']].values:   # 초미세먼지 데이터에서 시간(에서 분,초 뺀 것), 미세먼지만 떼어 냄
    tlist.append([i[0],i[1]])

노원행정동11_초미세먼지 = pd.DataFrame(tlist, columns = ['temp', 'c_mise_med'])
노원행정동11_초미세먼지['c_mise_med'].replace([-999,-9999], np.NaN, inplace=True)


# 초미세먼지 중앙값 구하기
노원행정동11_Cmedian = 노원행정동11_초미세먼지.pivot_table(index='temp',
                           aggfunc='median', 
                           values='c_mise_med')


no11_1 = pd.DataFrame(노원행정동11_Cmedian)


# 초미세먼지 최빈값 구하기 
gugan = []
for i in 노원행정동11[['temp', 'pm25']].values:
     gugan.append([i[0],[(i[1]//10)*10]])


gugan_11 = pd.DataFrame(gugan, columns = ['temp', 'gugan']).groupby('temp')['gugan'].sum()
gugan_11.head()


# 전체 데이터에 적용
cmise_mode = []
for i in range(len(gugan_11)):
  cmise_mode.append(mode(gugan_11.values[i]))


# 하나의 데이터프레임으로 만들기
no11.reset_index(inplace=True)
no11_1.reset_index(inplace=True)

all_no11 = pd.merge(no11[['temp', 'mise_med']], no11_1[['temp','c_mise_med']], on = 'temp')

all_no11.columns = [['중계2,3동','중계2,3동','중계2,3동'],['temp','mise_med','c_mise_med']]

all_no11['mise_mode'] = pd.DataFrame(mise_mode)

all_no11['c_mise_mode'] = pd.DataFrame(cmise_mode)

all_no11.columns = ['temp','mise_med','c_mise_med','mise_mode','c_mise_mode']

all_no11['행정동명'] = '중계2,3동'

all_no11.head()



## 12. 중계본동(station 1개)

In [0]:
# 행정동 - nowon12(중계본동) - station 1개
filepath = 'C:\Python\JupyterNotebook\[공모전]빅콘\DATASET(미세먼지)\환경기상데이터\노원구\{}.csv'\
.format(n_hdong.loc[no_dong[11]]['스테이션.1'])
노원행정동12 = pd.read_csv(filepath)
노원행정동12.drop(['Unnamed: 10','co2','vocs', 'flag', 'noise'],axis=1, inplace=True)
노원행정동12['DONG_NM'] = no_dong[11]
노원행정동12.head()

In [0]:
# 행정동별 datetime변경 및 이상치 처리 
# 행정동 - nowon12(중계본동) - station 1개

# tm 변수를 시간 변수로 바꾸고 인덱스로 설정해주기
노원행정동12['tm'] = pd.to_datetime(노원행정동12['tm'], format = '%Y%m%d%H%M')

#하고 싶은 것 :   
#동별 시간대별 미세먼지 농도
# -999, -9999 na로 바꾸고 해보자

노원행정동12['pm10'].replace([-999,-9999], np.NaN, inplace=True)

노원행정동12['pm25'].replace([-999,-9999], np.NaN, inplace=True)   

노원행정동12.describe(include='all')

# 시간데이터 분, 초만 잘라내서 'temp'라는 이름의 변수로 넣어줌

temp = [str(i)[:-6] for i in 노원행정동12.tm]
노원행정동12['temp'] = temp


# 미세먼지
# 같은 동 내 다른 스테이션에서 측정된 미세먼지농도를 하나의 리스트로 묶은 것 'mise'라는 이름의 변수를 가지는 데이터프레임 생성

tlist = []
for i in 노원행정동12[['temp','pm10']].values:   # 미세먼지 데이터에서 시간(에서 분,초 뺀 것), 미세먼지만 떼어 냄
    tlist.append([i[0],i[1]])

노원행정동12_미세먼지 = pd.DataFrame(tlist, columns = ['temp', 'mise_med'])
노원행정동12_미세먼지.head()

# 미세먼지 중앙값 구하기
노원행정동12_median = 노원행정동12_미세먼지.pivot_table(index='temp',
                           aggfunc='median', 
                           values='mise_med')

no12 = pd.DataFrame(노원행정동12_median)

# 미세먼지 최빈값 구하기 
# 위와 같은 일을 하는데 1의자리에서 버린 값을 넣어줌 (최빈구간 구하기 위해)

gugan = []
for i in 노원행정동12[['temp', 'pm10']].values:
     gugan.append([i[0],[(i[1]//10)*10]])


gugan_12 = pd.DataFrame(gugan, columns = ['temp', 'gugan']).groupby('temp')['gugan'].sum()
gugan_12.head()

# 전체 데이터에 적용
mise_mode = []
for i in range(len(gugan_12)):
  mise_mode.append(mode(gugan_12.values[i]))


# 초미세먼지
# 같은 동 내 다른 스테이션에서 측정된 미세먼지농도를 하나의 리스트로 묶은 것 'mise'라는 이름의 변수를 가지는 데이터프레임 생성

tlist = []
for i in 노원행정동12[['temp','pm25']].values:   # 초미세먼지 데이터에서 시간(에서 분,초 뺀 것), 미세먼지만 떼어 냄
    tlist.append([i[0],i[1]])

노원행정동12_초미세먼지 = pd.DataFrame(tlist, columns = ['temp', 'c_mise_med'])
노원행정동12_초미세먼지['c_mise_med'].replace([-999,-9999], np.NaN, inplace=True)

# 초미세먼지 중앙값 구하기  
노원행정동12_Cmedian = 노원행정동12_초미세먼지.pivot_table(index='temp',
                           aggfunc='median', 
                           values='c_mise_med')


no12_1 = pd.DataFrame(노원행정동12_Cmedian)

# 초미세먼지 최빈값 구하기 
gugan = []
for i in 노원행정동12[['temp', 'pm25']].values:
     gugan.append([i[0],[(i[1]//10)*10]])


gugan_12 = pd.DataFrame(gugan, columns = ['temp', 'gugan']).groupby('temp')['gugan'].sum()
gugan_12.head()


# 전체 데이터에 적용
cmise_mode = []
for i in range(len(gugan_12)):
  cmise_mode.append(mode(gugan_12.values[i]))


# 하나의 데이터프레임으로 만들기
no12.reset_index(inplace=True)
no12_1.reset_index(inplace=True)

all_no12 = pd.merge(no12[['temp', 'mise_med']], no12_1[['temp','c_mise_med']], on = 'temp')

all_no12.columns = [['중계본동','중계본동','중계본동'],['temp','mise_med','c_mise_med']]

all_no12['mise_mode'] = pd.DataFrame(mise_mode)

all_no12['c_mise_mode'] = pd.DataFrame(cmise_mode)

all_no12.columns = ['temp','mise_med','c_mise_med','mise_mode','c_mise_mode']

all_no12['행정동명'] = '중계본동'

all_no12.head()



## 13. 하계1동(station 2개)

In [0]:
# 행정동 - nowon13(하계1동) - station 2개
노원행정동13 = multi_st(12)
노원행정동13.head()

In [0]:
# tm 변수를 시간 변수로 바꾸고 인덱스로 설정해주기

노원행정동13['tm'] = pd.to_datetime(노원행정동13['tm'], format = '%Y%m%d%H%M')

#하고 싶은 것 :   
#동별 시간대별 미세먼지 농도
# -999, -9999 na로 바꾸고 해보자

노원행정동13['pm10_x'].replace([-999,-9999], np.NaN, inplace=True)
노원행정동13['pm10_y'].replace([-999,-9999], np.NaN, inplace=True)

노원행정동13['pm25_x'].replace([-999,-9999], np.NaN, inplace=True)   
노원행정동13['pm25_y'].replace([-999,-9999], np.NaN, inplace=True)   

노원행정동13.describe(include='all')

# 시간데이터 분, 초만 잘라내서 'temp'라는 이름의 변수로 넣어줌

temp = [str(i)[:-6] for i in 노원행정동13.tm]
노원행정동13['temp'] = temp


# 미세먼지
# 같은 동 내 다른 스테이션에서 측정된 미세먼지농도를 하나의 리스트로 묶은 것 'mise'라는 이름의 변수를 가지는 데이터프레임 생성

tlist = []
for i in 노원행정동13[['temp','pm10_x', 'pm10_y']].values:   # 미세먼지 데이터에서 시간(에서 분,초 뺀 것), 미세먼지만 떼어 냄
    tlist.append([i[0],[i[1], i[2]]])

    
tdataframe = pd.DataFrame(tlist, columns = ['temp', 'mise'])
tdataframe.head()
no13 = pd.DataFrame(tdataframe.groupby('temp')['mise'].sum())


# 미세먼지 중앙값 구하기

mise_med = []

for i in no1.values:
    mise_med.append(s.median(i[0]))
no13['mise_med'] = mise_med


# 미세먼지 최빈값 구하기 
# 위와 같은 일을 하는데 1의자리에서 버린 값을 넣어줌 (최빈구간 구하기 위해)

gugan = []
for i in 노원행정동13[['temp', 'pm10_x', 'pm10_y']].values:
     gugan.append([i[0],[(i[1]//10)*10, (i[2]//10)*10]])


gugan_13 = pd.DataFrame(gugan, columns = ['temp', 'gugan']).groupby('temp')['gugan'].sum()
gugan_13.head()


# 전체 데이터에 적용
mise_mode = []
for i in range(len(gugan_13)):
  mise_mode.append(mode(gugan_13.values[i]))


# 초미세먼지
# 같은 동 내 다른 스테이션에서 측정된 미세먼지농도를 하나의 리스트로 묶은 것 'mise'라는 이름의 변수를 가지는 데이터프레임 생성

tlist = []
for i in 노원행정동13[['temp','pm25_x', 'pm25_y']].values:   # 초미세먼지 데이터에서 시간(에서 분,초 뺀 것), 미세먼지만 떼어 냄
    tlist.append([i[0],[i[1], i[2]]])

tdataframe = pd.DataFrame(tlist, columns = ['temp', 'c_mise'])
tdataframe.head()
no13_1 = pd.DataFrame(tdataframe.groupby('temp')['c_mise'].sum())

# 초미세먼지 중앙값 구하기

mise_med = []

for i in no13_1.values:
    mise_med.append(s.median(i[0]))
no13_1['c_mise_med'] = mise_med


# 초미세먼지 최빈값 구하기
# 위와 같은 일을 하는데 1의자리에서 버린 값을 넣어줌 (최빈구간 구하기 위해)

gugan = []
for i in 노원행정동13[['temp','pm25_x', 'pm25_y']].values:
    gugan.append([i[0],[(i[1]//10)*10, (i[2]//10)*10]])
    
gugan_13 = pd.DataFrame(gugan, columns = ['temp', 'gugan']).groupby('temp')['gugan'].sum()
gugan_13

# 전체 데이터에 적용
cmise_mode = []
for i in range(len(gugan_13)):
  cmise_mode.append(mode(gugan_13.values[i]))

# 하나의 데이터프레임으로 만들기

no13.reset_index(inplace=True)
no13_1.reset_index(inplace=True)

all_no13 = pd.merge(no13[['temp', 'mise_med']], no13_1[['temp','c_mise_med']], on = 'temp')

all_no13.columns = [['하계1동','하계1동','하계1동'],['temp','mise_med','c_mise_med']]

all_no13['mise_mode'] = pd.DataFrame(mise_mode)

all_no13['c_mise_mode'] = pd.DataFrame(cmise_mode)

all_no13.columns = ['temp','mise_med','c_mise_med','mise_mode','c_mise_mode']

all_no13['행정동명'] = '하계1동'


all_no13.head()


In [0]:
all_no1.head()

In [0]:
all_no2.head()

In [0]:
all_no3.head()

In [0]:
all_no4.head()

In [0]:
all_no5.head()

In [0]:
all_no6.head()

In [0]:
all_no7.head()

In [0]:
all_no8.head()

In [0]:
all_no9.head()

In [0]:
all_no10.head()

In [0]:
all_no11.head()

In [0]:
all_no12.head()

In [0]:
all_no13.head()

In [0]:
노원구 = pd.concat([all_no1,all_no2,all_no3,all_no4,all_no5,all_no6,
                all_no7,all_no8,all_no9,all_no10,all_no11,all_no12,all_no13] , axis=0, ignore_index = True)

In [0]:
import matplotlib as mpl

# 한글 깨짐 방지
mpl.rcParams['font.family'].insert(0,'Malgun Gothic')

# MacOS인 경우에는 AppleGothic

In [0]:
import matplotlib.pyplot as plt

plt.figure()

plt.plot(all_no1.temp, all_no1.mise_med)

### 2. airkorea 전처리 파일 하나로 합치기(18년 4월~ 19년 2월)

In [0]:
#18년도 airkorea 파일 하나로 합치기

months_18 = range(4,13)

data = []
#columns = ['Unnamed: 0', '시도', '측정소명','측정소코드','아황산가스','일산화탄소','오존','이산화질소','PM10','PM2.5','날짜']

for month in months_18:
    path = '환경기상데이터/airkorea_미세먼지/excel/2018 %d월 종로 노원.xls' % month
    frame = pd.read_excel(path)
    frame = frame[frame['측정소명']=='노원구']
    
    frame['월'] = month
    data.append(frame)

airkorea_18 = pd.concat(data, ignore_index=True)

#위의 결과에 19년도 airkorea 파일 추가하기

months_19 = range(1,3)

for month in months_19:
    path = '환경기상데이터/airkorea_미세먼지/excel/2019 %d월 종로 노원.xls' % month
    frame = pd.read_excel(path)
    frame = frame[frame['측정소명']=='노원구']
    
    frame['월'] = month
    data.append(frame)

airkorea_18_19 = pd.concat(data, ignore_index=True)

airkorea_18_19

In [0]:
mise = pd.DataFrame(airkorea_18_19[['날짜','PM10','PM2.5']])

In [0]:
mise.info()

# 결측치 존재
# PM10 : 190개
# PM2.5 : 120개 

In [0]:
mise.head()

In [0]:
all_no1.info()

In [0]:
all_no1.head()

### 3. 데이터 합치기

In [0]:
"""
nums = range(1,14)
pieces = []

for num in nums:

    file = all_no%d, % num
    data = pd.merge(file, mise, left_on='temp', right_on='날짜', how='outer')

    pieces.append(data)
    

"""
all_no1_add_airforce = pd.merge(all_no1, mise, left_on='temp', right_on='날짜', how='outer')
all_no2_add_airforce = pd.merge(all_no2, mise, left_on='temp', right_on='날짜', how='outer')
all_no3_add_airforce = pd.merge(all_no3, mise, left_on='temp', right_on='날짜', how='outer')
all_no4_add_airforce = pd.merge(all_no4, mise, left_on='temp', right_on='날짜', how='outer')
all_no5_add_airforce = pd.merge(all_no5, mise, left_on='temp', right_on='날짜', how='outer')
all_no6_add_airforce = pd.merge(all_no6, mise, left_on='temp', right_on='날짜', how='outer')
all_no7_add_airforce = pd.merge(all_no7, mise, left_on='temp', right_on='날짜', how='outer')
all_no8_add_airforce = pd.merge(all_no8, mise, left_on='temp', right_on='날짜', how='outer')
all_no9_add_airforce = pd.merge(all_no9, mise, left_on='temp', right_on='날짜', how='outer')
all_no10_add_airforce = pd.merge(all_no10, mise, left_on='temp', right_on='날짜', how='outer')
all_no11_add_airforce = pd.merge(all_no11, mise, left_on='temp', right_on='날짜', how='outer')
all_no12_add_airforce = pd.merge(all_no12, mise, left_on='temp', right_on='날짜', how='outer')
all_no13_add_airforce = pd.merge(all_no13, mise, left_on='temp', right_on='날짜', how='outer')
"""
#if no1_all_add_4['mise_med'].isnull():
#    no1_all_add_4.mise_med.replace(np,NaN, no1_all_add_4.PM10)
#    #no1_all_add_4.mise_mode.fillna(no1_all_add_4.PM10)
"""

### 4. 원본 데이터가 NA값인 경우 airkorea값을 채워주기

In [0]:
all_no1_add_airforce.head()

In [0]:
all_no2_add_airforce.head()

In [0]:
all_no3_add_airforce.head()

In [0]:
all_no4_add_airforce.head()

In [0]:
all_no5_add_airforce.head()

In [0]:
all_no6_add_airforce.head()

In [0]:
all_no7_add_airforce.head()

In [0]:
all_no8_add_airforce.head()

In [0]:
all_no9_add_airforce.head()

In [0]:
all_no10_add_airforce.head()

In [0]:
all_no11_add_airforce.head()

In [0]:
all_no12_add_airforce.head()

In [0]:
all_no13_add_airforce.head()

In [0]:
all_no1_add_airforce['mise_mode'] = all_no1_add_airforce['mise_mode'].astype(float)
all_no1_add_airforce['c_mise_mode'] = all_no1_add_airforce['c_mise_mode'].astype(float)

all_no2_add_airforce['mise_mode'] = all_no2_add_airforce['mise_mode'].astype(float)
all_no2_add_airforce['c_mise_mode'] = all_no2_add_airforce['c_mise_mode'].astype(float)

all_no3_add_airforce['mise_mode'] = all_no3_add_airforce['mise_mode'].astype(float)
all_no3_add_airforce['c_mise_mode'] = all_no3_add_airforce['c_mise_mode'].astype(float)

all_no4_add_airforce['mise_mode'] = all_no4_add_airforce['mise_mode'].astype(float)
all_no4_add_airforce['c_mise_mode'] = all_no4_add_airforce['c_mise_mode'].astype(float)

all_no5_add_airforce['mise_mode'] = all_no5_add_airforce['mise_mode'].astype(float)
all_no5_add_airforce['c_mise_mode'] = all_no5_add_airforce['c_mise_mode'].astype(float)

all_no6_add_airforce['mise_mode'] = all_no6_add_airforce['mise_mode'].astype(float)
all_no6_add_airforce['c_mise_mode'] = all_no6_add_airforce['c_mise_mode'].astype(float)

all_no7_add_airforce['mise_mode'] = all_no7_add_airforce['mise_mode'].astype(float)
all_no7_add_airforce['c_mise_mode'] = all_no7_add_airforce['c_mise_mode'].astype(float)

all_no8_add_airforce['mise_mode'] = all_no8_add_airforce['mise_mode'].astype(float)
all_no8_add_airforce['c_mise_mode'] = all_no8_add_airforce['c_mise_mode'].astype(float)

all_no9_add_airforce['mise_mode'] = all_no9_add_airforce['mise_mode'].astype(float)
all_no9_add_airforce['c_mise_mode'] = all_no9_add_airforce['c_mise_mode'].astype(float)

all_no10_add_airforce['mise_mode'] = all_no10_add_airforce['mise_mode'].astype(float)
all_no10_add_airforce['c_mise_mode'] = all_no10_add_airforce['c_mise_mode'].astype(float)

all_no11_add_airforce['mise_mode'] = all_no11_add_airforce['mise_mode'].astype(float)
all_no11_add_airforce['c_mise_mode'] = all_no11_add_airforce['c_mise_mode'].astype(float)

all_no12_add_airforce['mise_mode'] = all_no12_add_airforce['mise_mode'].astype(float)
all_no12_add_airforce['c_mise_mode'] = all_no12_add_airforce['c_mise_mode'].astype(float)

all_no13_add_airforce['mise_mode'] = all_no13_add_airforce['mise_mode'].astype(float)
all_no13_add_airforce['c_mise_mode'] = all_no13_add_airforce['c_mise_mode'].astype(float)

In [0]:
"""
nums = range(1,14)

for num in nums: 
    all_no + num + _add_airforce['mise_mode'] = all_no+num+_add_airforce['mise_mode'].astype(float)
    all_no + num + _add_airforce['c_mise_mode'] = all_no+num_add_airforce['c_mise_mode'].astype(float)

for num in nums:
    all_no+num+_add_airforce.mise_med.fillna(all_no+num+_add_airforce.PM10, inplace=True)
    all_no+num+_add_airforce.mise_mode.fillna(all_no+num+_add_airforce.PM10, inplace=True)
    all_no+num+_add_airforce.c_mise_med.fillna(all_no+num+_add_airforce['PM2.5'], inplace=True)
    all_no+num+_add_airforce.c_mise_mode.fillna(all_no+num+_add_airforce['PM2.5'], inplace=True)
"""

all_no1_add_airforce.mise_med.fillna(all_no1_add_airforce.PM10, inplace=True)
all_no1_add_airforce.mise_mode.fillna(all_no1_add_airforce.PM10, inplace=True)
all_no1_add_airforce.c_mise_med.fillna(all_no1_add_airforce['PM2.5'], inplace=True)
all_no1_add_airforce.c_mise_mode.fillna(all_no1_add_airforce['PM2.5'], inplace=True)

all_no2_add_airforce.mise_med.fillna(all_no2_add_airforce.PM10, inplace=True)
all_no2_add_airforce.mise_mode.fillna(all_no2_add_airforce.PM10, inplace=True)
all_no2_add_airforce.c_mise_med.fillna(all_no2_add_airforce['PM2.5'], inplace=True)
all_no2_add_airforce.c_mise_mode.fillna(all_no2_add_airforce['PM2.5'], inplace=True)

all_no3_add_airforce.mise_med.fillna(all_no3_add_airforce.PM10, inplace=True)
all_no3_add_airforce.mise_mode.fillna(all_no3_add_airforce.PM10, inplace=True)
all_no3_add_airforce.c_mise_med.fillna(all_no3_add_airforce['PM2.5'], inplace=True)
all_no3_add_airforce.c_mise_mode.fillna(all_no3_add_airforce['PM2.5'], inplace=True)

all_no4_add_airforce.mise_med.fillna(all_no4_add_airforce.PM10, inplace=True)
all_no4_add_airforce.mise_mode.fillna(all_no4_add_airforce.PM10, inplace=True)
all_no4_add_airforce.c_mise_med.fillna(all_no4_add_airforce['PM2.5'], inplace=True)
all_no4_add_airforce.c_mise_mode.fillna(all_no4_add_airforce['PM2.5'], inplace=True)

all_no5_add_airforce.mise_med.fillna(all_no5_add_airforce.PM10, inplace=True)
all_no5_add_airforce.mise_mode.fillna(all_no5_add_airforce.PM10, inplace=True)
all_no5_add_airforce.c_mise_med.fillna(all_no5_add_airforce['PM2.5'], inplace=True)
all_no5_add_airforce.c_mise_mode.fillna(all_no5_add_airforce['PM2.5'], inplace=True)

all_no6_add_airforce.mise_med.fillna(all_no6_add_airforce.PM10, inplace=True)
all_no6_add_airforce.mise_mode.fillna(all_no6_add_airforce.PM10, inplace=True)
all_no6_add_airforce.c_mise_med.fillna(all_no6_add_airforce['PM2.5'], inplace=True)
all_no6_add_airforce.c_mise_mode.fillna(all_no6_add_airforce['PM2.5'], inplace=True)


all_no7_add_airforce.mise_med.fillna(all_no7_add_airforce.PM10, inplace=True)
all_no7_add_airforce.mise_mode.fillna(all_no7_add_airforce.PM10, inplace=True)
all_no7_add_airforce.c_mise_med.fillna(all_no7_add_airforce['PM2.5'], inplace=True)
all_no7_add_airforce.c_mise_mode.fillna(all_no7_add_airforce['PM2.5'], inplace=True)

all_no8_add_airforce.mise_med.fillna(all_no8_add_airforce.PM10, inplace=True)
all_no8_add_airforce.mise_mode.fillna(all_no8_add_airforce.PM10, inplace=True)
all_no8_add_airforce.c_mise_med.fillna(all_no8_add_airforce['PM2.5'], inplace=True)
all_no8_add_airforce.c_mise_mode.fillna(all_no8_add_airforce['PM2.5'], inplace=True)

all_no9_add_airforce.mise_med.fillna(all_no9_add_airforce.PM10, inplace=True)
all_no9_add_airforce.mise_mode.fillna(all_no9_add_airforce.PM10, inplace=True)
all_no9_add_airforce.c_mise_med.fillna(all_no9_add_airforce['PM2.5'], inplace=True)
all_no9_add_airforce.c_mise_mode.fillna(all_no9_add_airforce['PM2.5'], inplace=True)

all_no10_add_airforce.mise_med.fillna(all_no10_add_airforce.PM10, inplace=True)
all_no10_add_airforce.mise_mode.fillna(all_no10_add_airforce.PM10, inplace=True)
all_no10_add_airforce.c_mise_med.fillna(all_no10_add_airforce['PM2.5'], inplace=True)
all_no10_add_airforce.c_mise_mode.fillna(all_no10_add_airforce['PM2.5'], inplace=True)

all_no11_add_airforce.mise_med.fillna(all_no11_add_airforce.PM10, inplace=True)
all_no11_add_airforce.mise_mode.fillna(all_no11_add_airforce.PM10, inplace=True)
all_no11_add_airforce.c_mise_med.fillna(all_no11_add_airforce['PM2.5'], inplace=True)
all_no11_add_airforce.c_mise_mode.fillna(all_no11_add_airforce['PM2.5'], inplace=True)

all_no12_add_airforce.mise_med.fillna(all_no12_add_airforce.PM10, inplace=True)
all_no12_add_airforce.mise_mode.fillna(all_no12_add_airforce.PM10, inplace=True)
all_no12_add_airforce.c_mise_med.fillna(all_no12_add_airforce['PM2.5'], inplace=True)
all_no12_add_airforce.c_mise_mode.fillna(all_no12_add_airforce['PM2.5'], inplace=True)

all_no13_add_airforce.mise_med.fillna(all_no13_add_airforce.PM10, inplace=True)
all_no13_add_airforce.mise_mode.fillna(all_no13_add_airforce.PM10, inplace=True)
all_no13_add_airforce.c_mise_med.fillna(all_no13_add_airforce['PM2.5'], inplace=True)
all_no13_add_airforce.c_mise_mode.fillna(all_no13_add_airforce['PM2.5'], inplace=True)


In [0]:
all_no1_add_airforce.info()

In [0]:
all_no1_add_airforce.tail()

In [0]:
all_no2_add_airforce.info()

In [0]:
all_no3_add_airforce.info()

In [0]:
all_no4_add_airforce.info()

In [0]:
all_no5_add_airforce.info()

In [0]:
all_no1_add_airforce.to_excel('no1.xlsx' ,sheet_name='no1')

In [0]:
all_no2_add_airforce.to_excel('no2.xlsx' ,sheet_name='no2')

In [0]:
all_no3_add_airforce.to_excel('no3.xlsx' ,sheet_name='no3')

In [0]:
all_no4_add_airforce.to_excel('no4.xlsx' ,sheet_name='no4')

In [0]:
all_no5_add_airforce.to_excel('no5.xlsx' ,sheet_name='no5')

In [0]:
all_no6_add_airforce.to_excel('no6.xlsx' ,sheet_name='no6')

In [0]:
all_no7_add_airforce.to_excel('no7.xlsx' ,sheet_name='no7')

In [0]:
all_no8_add_airforce.to_excel('no8.xlsx' ,sheet_name='no8')

In [0]:
all_no9_add_airforce.to_excel('no9.xlsx' ,sheet_name='no9')

In [0]:
all_no10_add_airforce.to_excel('no10.xlsx' ,sheet_name='no10')

In [0]:
all_no11_add_airforce.to_excel('no11.xlsx' ,sheet_name='no11')

In [0]:
all_no12_add_airforce.to_excel('no12.xlsx' ,sheet_name='no12')

In [0]:
all_no13_add_airforce.to_excel('no13.xlsx' ,sheet_name='no13')

In [0]:
all_no1.to_excel('공릉1동.xlsx')

In [0]:
all_no2.to_excel('공릉2동.xlsx')

In [0]:
all_no3.to_excel('상계1동.xlsx')

In [0]:
all_no4.to_excel('상계2동.xlsx')

In [0]:
all_no5.to_excel('상계3,4동.xlsx')

In [0]:
all_no6.to_excel('상계5동.xlsx')

In [0]:
all_no7.to_excel('상계6,7동.xlsx')

In [0]:
all_no8.to_excel('상계10동.xlsx')

In [0]:
all_no9.to_excel('월계1동.xlsx')

In [0]:
all_no10.to_excel('월계3동.xlsx')

In [0]:
all_no11.to_excel('중계2,3동.xlsx')

In [0]:
all_no12.to_excel('중계본동.xlsx')

In [0]:
all_no13.to_excel('하계1동.xlsx')